In [12]:
from markitdown import MarkItDown
from docx import Document
import pypandoc

In [14]:
test_doc = Document("стратегия_2035.docx")

In [35]:
structure_dict = {}
current_par = []
privios_par = None
for paragraph in test_doc.paragraphs:
    if paragraph.style.name == "ConsPlusTitle":
        current_par.append(paragraph)

AttributeError: 'list' object has no attribute 'style'

In [30]:
structure_dict

{'ПРИЛОЖЕНИЕ': None,
 'к Закону Санкт-Петербурга': None,
 '"О Стратегии': None,
 'социально-экономического': None,
 'развития Санкт-Петербурга': None,
 'на период до 2035 года"': None,
 'от 19.12.2018 N 771-164': None,
 '': 'реализации Стратегии 2035',
 'Стратегия социально-экономического развития Санкт-Петербурга на период до 2035 года (далее - Стратегия 2035) разработана в целях определения приоритетов, целей и задач социально-экономического развития Санкт-Петербурга, согласованных с приоритетами и целями социально-экономического развития Российской Федерации.': '1. Общие положения',
 'Система приоритетов, направлений, целей и задач социально-экономической политики Санкт-Петербурга, определенных в Стратегии 2035, является ориентиром долгосрочного развития Санкт-Петербурга.': '1. Общие положения',
 'Основные понятия, используемые в Стратегии 2035, применяются в значениях, определенных действующим законодательством.': '1. Общие положения',
 'Приоритеты и цели социально-экономического р